<a href="https://colab.research.google.com/github/CesarChaMal/ollama-webui/blob/main/ollama-webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Linux uninstall ollama
!rm -rf *
!echo $OLLAMA_HOST
!export OLLAMA_HOST=http://localhost:11434
import os
os.environ['OLLAMA_HOST'] = 'http://localhost:11434'
!echo $OLLAMA_HOST
!command -v systemctl >/dev/null && ps aux | grep 'ollama' | grep -v grep | awk '{print $2}' | xargs -r kill
!rm /etc/systemd/system/ollama.service
!sudo rm $(which ollama)
!sudo rm -r /usr/share/ollama
!sudo userdel ollama

# Download and run the Ollama Linux install script
!curl -fsSL https://ollama.com/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
#!ollama start
!nohup ollama start &
!nohup ollama serve &
!ps aux | grep ollama
!journalctl -u ollama.service



http://localhost:11434
rm: cannot remove '/etc/systemd/system/ollama.service': No such file or directory
rm: missing operand
Try 'rm --help' for more information.
rm: cannot remove '/usr/share/ollama': No such file or directory
userdel: user 'ollama' does not exist
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100  8354    0  8354    0     0  23619      0 --:--:-- --:--:-- --:--:-- 23665
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 0.0.0.0:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 http://security.ubun

In [ ]:
!rm -rf *
#!command -v systemctl
#!ps aux | grep '8000'
#!sudo apt-get install -y net-tools
#!lsof -i :8000
#!lsof -i :11434
#!netstat -ltnp | grep :8080
#!netstat -ltnp | grep :11434
#!ps -aef
#!command -v systemctl >/dev/null && ps aux | grep '8000' | grep -v grep | awk '{print $2}' | xargs -r kill
#!lsof -i :800 -t | xargs -r kill
#!sudo netstat -ltnp | grep ':8000' | awk '{print $7}' | cut -d'/' -f1 | xargs -r kill

!pip install aiohttp pyngrok
!pip show pyngrok
!pip install --upgrade pyngrok

import os
import asyncio
from aiohttp import ClientSession
from pyngrok import ngrok

# Set LD_LIBRARY_PATH so the system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run(cmd):
    '''
    run is a helper function to run subcommands asynchronously.
    '''
    print('>>> starting', *cmd)
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )


#await asyncio.gather(
#    run(['ollama', 'serve']),
#    run(['ngrok', 'http', '--log', 'stderr', '11434']),
#)


async def run_ollama_commands():
    # Define the commands you want to run
    commands = [
        ['ollama', 'list'],
        ['ollama', 'run', 'mistral']
    ]

    for cmd in commands:
        print('>>> Executing:', ' '.join(cmd))
        process = await asyncio.create_subprocess_exec(*cmd, stdout=asyncio.subprocess.PIPE,
                                                       stderr=asyncio.subprocess.PIPE)

        # Capture the output
        stdout, stderr = await process.communicate()
        if stdout:
            print(stdout.decode())
        if stderr:
            print(stderr.decode())


# Run the ollama commands
await run_ollama_commands()
#ngrok.disconnect(public_url.public_url)


import asyncio

async def run_shell_command(cmd):
    print('>>> Executing:', cmd)
    process = await asyncio.create_subprocess_shell(
        cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    stdout, stderr = await process.communicate()
    if stdout:
        print(stdout.decode())
    if stderr:
        print(stderr.decode())

async def run_shell_commands_sequentially(commands):
    for cmd in commands:
        await run_shell_command(cmd)


commands = [
    'curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -',
    'sudo apt-get install -y nodejs',
    'node -v',
    'npm -v',
    'git clone https://github.com/CesarChaMal/ollama-webui.git',
    'mv ollama-webui/* .',
    'cp -RPp example.env .env',
    'cat .env',
    'npm install vite',
    'npm install',
    'npm run build',
    'pip install uvicorn',
    'pip install --ignore-installed blinker',
    'pip install kaleido',
    'pip install openai',
    'pip install cohere',
    'pip install tiktoken',
    'pip install -r backend/requirements.txt -U',
]
await run_shell_commands_sequentially(commands)

#!sed -i 's/^OPENAI_API_KEY=.*/OPENAI_API_KEY=your_openai_api_key_here/' .env
#!sed -i 's/^NGROK_AUTHTOKEN=.*/NGROK_AUTHTOKEN=your_ngrok_auth_token_here/' .env

import subprocess

# Replace these with your actual keys
openai_api_key = "your_openai_api_key_here"
ngrok_authtoken = "your_ngrok_auth_token_here"

# Construct the sed commands
sed_openai = f"sed -i 's/^OPENAI_API_KEY=.*/OPENAI_API_KEY={openai_api_key}/' .env"
sed_ngrok = f"sed -i 's/^NGROK_AUTHTOKEN=.*/NGROK_AUTHTOKEN={ngrok_authtoken}/' .env"

# Execute the commands
subprocess.run(sed_openai, shell=True)
subprocess.run(sed_ngrok, shell=True)

#uvicorn_command = "uvicorn backend.main:app --reload --host 0.0.0.0 --port 8000 --forwarded-allow-ips '*'"
#await run_shell_command(uvicorn_command)

# Setting Ollama Web UI
#!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
#!sudo apt-get install -y nodejs
#!node -v
#!npm -v

#!git clone https://github.com/CesarChaMal/ollama-webui.git
#!mv ollama-webui/* .

#!cp -RPp example.env .env
#!cat .env

#!npm install vite
#!npm install
#!npm run build

#!pip install uvicorn
#!pip install --ignore-installed blinker
#!pip install kaleido
#!pip install openai
#!pip install cohere
#!pip install tiktoken
#!pip install -r backend/requirements.txt -U
#!uvicorn backend.main:app --reload --host 0.0.0.0 --port 8000 --forwarded-allow-ips '*'
!nohup uvicorn backend.main:app --reload --host 0.0.0.0 --port 8000 --forwarded-allow-ips '*' &

from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the NGROK_AUTHTOKEN
ngrok_token = os.getenv('NGROK_AUTHTOKEN')
openai_key = os.getenv('OPENAI_API_KEY')

# Use ngrok_token in your application
#print(ngrok_token)

# Set up ngrok
ngrok.set_auth_token(ngrok_token)
#public_url = ngrok.connect(11434, "http")
public_url = ngrok.connect(8000, "http")
print("Ngrok Tunnel URL:", public_url)
!cat /tmp/ngrok.log
!cat nohup.out

# Extract the actual URL from the ngrok object
actual_url = str(public_url).strip()

# Print the actual URL to verify
print("Actual URL:", actual_url)

!pip install Pygments
!curl http://localhost:11434 | pygmentize -l console
!curl http://0.0.0.0:8000 | pygmentize -l console

# Use the actual URL in the curl command
os.system(f"curl {actual_url} | pygmentize -l console")



Name: pyngrok
Version: 7.0.5
Summary: A Python wrapper for ngrok.
Home-page: https://github.com/alexdlaird/pyngrok
Author: Alex Laird
Author-email: contact@alexlaird.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: PyYAML
Required-by: 
>>> Executing: ollama list
NAME	ID	SIZE	MODIFIED 

>>> Executing: ollama run mistral
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling e8a35b5937a5...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕          

0

The previous cell starts two processes, `ollama` and `ngrok`. The log output will show a line like the following which describes the external address.

```
t=2023-11-12T22:55:56+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://8249-34-125-179-11.ngrok.io
```

The external address in this case is `https://8249-34-125-179-11.ngrok.io` which can be passed into `OLLAMA_HOST` to access this instance.

```bash
export OLLAMA_HOST=https://8249-34-125-179-11.ngrok.io
ollama list
ollama run mistral
```

In [ ]:
!ollama list
!nohup ollama run deepseek-coder &
!nohup ollama run llava &
!nohup ollama run wizard-vicuna-uncensored &


NAME          	ID          	SIZE  	MODIFIED       
mistral:latest	61e88e884507	4.1 GB	13 minutes ago	
pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling d040cc185215...   0% ▕▏    0 B/776 MB                  pulling manifest 
pulling d040cc185215...   0% ▕▏    0 B/776 MB                  pulling manifest 
pulling d040cc185215...   0% ▕▏    0 B/776 MB                  pulling manifest 
pulling d040cc185215...   0% ▕▏    0 B/776 MB                  pulling manifest 
pulling d040cc185215...   0% ▕▏    0 B/776 MB                  pulling manifest 
